In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("IRIS.csv")

# Encode species
le = LabelEncoder()
df['species_encoded'] = le.fit_transform(df['species'])

# Split features and labels
X = df.drop(['species', 'species_encoded'], axis=1)
y = df['species_encoded']

# Train-test split 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict
y_pred = rf.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
import joblib

joblib.dump(rf, "iris_random_forest_model.pkl")
joblib.dump(scaler, "iris_scaler.pkl")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 5))
sns.violinplot(x='species', y='petal_width', data=df, palette='pastel')
plt.title("Distribution of Petal Width per Iris Species")
plt.ylabel("Petal Width (cm)")
plt.xlabel("Species")
plt.tight_layout()
plt.savefig("violin_plot_petal_width.png")
plt.show()


In [ ]:
from matplotlib.colors import ListedColormap

# Use petal_length and petal_width only
X_vis = df[['petal_length', 'petal_width']].values
y_vis = df['species_encoded'].values

clf_vis = RandomForestClassifier(n_estimators=50, random_state=42)
clf_vis.fit(X_vis, y_vis)

# Mesh grid
x_min, x_max = X_vis[:, 0].min() - 1, X_vis[:, 0].max() + 1
y_min, y_max = X_vis[:, 1].min() - 1, X_vis[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))
Z = clf_vis.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot
plt.figure(figsize=(8, 6))
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ['#FF0000', '#00FF00', '#0000FF']

plt.contourf(xx, yy, Z, cmap=cmap_light, alpha=0.6)
sns.scatterplot(x=X_vis[:, 0], y=X_vis[:, 1], hue=df['species'],
                palette=cmap_bold, edgecolor='k', s=70)

plt.xlabel("Petal Length")
plt.ylabel("Petal Width")
plt.title("Decision Boundary - Random Forest (Petal Features)")
plt.tight_layout()
plt.savefig("decision_boundary.png")
plt.show()
